In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Load the data

In [2]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

### Tokenization
Create integer values corresponding to characters

In [3]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [4]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [5]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [6]:
# check if GPU is available
GPU = torch.cuda.is_available()
if(GPU):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


## Define the RNN

In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## Pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## Put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (GPU):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

#### Training Function

In [9]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(GPU):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(GPU):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(GPU):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [10]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [23]:
print(encoded.shape)

(1985223,)


In [13]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2513... Val Loss: 3.1564
Epoch: 1/20... Step: 20... Loss: 3.1483... Val Loss: 3.1273
Epoch: 1/20... Step: 30... Loss: 3.1429... Val Loss: 3.1219
Epoch: 1/20... Step: 40... Loss: 3.1102... Val Loss: 3.1192
Epoch: 1/20... Step: 50... Loss: 3.1419... Val Loss: 3.1171
Epoch: 1/20... Step: 60... Loss: 3.1161... Val Loss: 3.1146
Epoch: 1/20... Step: 70... Loss: 3.1053... Val Loss: 3.1123
Epoch: 1/20... Step: 80... Loss: 3.1169... Val Loss: 3.1054
Epoch: 1/20... Step: 90... Loss: 3.1056... Val Loss: 3.0890
Epoch: 1/20... Step: 100... Loss: 3.0681... Val Loss: 3.0499
Epoch: 1/20... Step: 110... Loss: 3.0479... Val Loss: 2.9894
Epoch: 1/20... Step: 120... Loss: 2.9209... Val Loss: 2.9224
Epoch: 1/20... Step: 130... Loss: 2.8683... Val Loss: 2.8262
Epoch: 2/20... Step: 140... Loss: 2.7812... Val Loss: 2.7267
Epoch: 2/20... Step: 150... Loss: 2.7785... Val Loss: 2.7195
Epoch: 2/20... Step: 160... Loss: 2.6761... Val Loss: 2.6357
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.4311... Val Loss: 1.4514
Epoch: 10/20... Step: 1360... Loss: 1.4411... Val Loss: 1.4510
Epoch: 10/20... Step: 1370... Loss: 1.4318... Val Loss: 1.4480
Epoch: 10/20... Step: 1380... Loss: 1.4717... Val Loss: 1.4456
Epoch: 10/20... Step: 1390... Loss: 1.4673... Val Loss: 1.4424
Epoch: 11/20... Step: 1400... Loss: 1.4699... Val Loss: 1.4393
Epoch: 11/20... Step: 1410... Loss: 1.4743... Val Loss: 1.4372
Epoch: 11/20... Step: 1420... Loss: 1.4739... Val Loss: 1.4316
Epoch: 11/20... Step: 1430... Loss: 1.4325... Val Loss: 1.4328
Epoch: 11/20... Step: 1440... Loss: 1.4621... Val Loss: 1.4316
Epoch: 11/20... Step: 1450... Loss: 1.3929... Val Loss: 1.4296
Epoch: 11/20... Step: 1460... Loss: 1.4260... Val Loss: 1.4325
Epoch: 11/20... Step: 1470... Loss: 1.4095... Val Loss: 1.4258
Epoch: 11/20... Step: 1480... Loss: 1.4255... Val Loss: 1.4257
Epoch: 11/20... Step: 1490... Loss: 1.4208... Val Loss: 1.4198
Epoch: 11/20... Step: 1500... Loss: 1.4003... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.2498... Val Loss: 1.2924
Epoch: 20/20... Step: 2670... Loss: 1.2505... Val Loss: 1.2911
Epoch: 20/20... Step: 2680... Loss: 1.2398... Val Loss: 1.2937
Epoch: 20/20... Step: 2690... Loss: 1.2434... Val Loss: 1.2927
Epoch: 20/20... Step: 2700... Loss: 1.2476... Val Loss: 1.2894
Epoch: 20/20... Step: 2710... Loss: 1.2070... Val Loss: 1.2876
Epoch: 20/20... Step: 2720... Loss: 1.2249... Val Loss: 1.2848
Epoch: 20/20... Step: 2730... Loss: 1.2118... Val Loss: 1.2839
Epoch: 20/20... Step: 2740... Loss: 1.2042... Val Loss: 1.2871
Epoch: 20/20... Step: 2750... Loss: 1.2083... Val Loss: 1.2903
Epoch: 20/20... Step: 2760... Loss: 1.2146... Val Loss: 1.2841
Epoch: 20/20... Step: 2770... Loss: 1.2472... Val Loss: 1.2806
Epoch: 20/20... Step: 2780... Loss: 1.2602... Val Loss: 1.2827


#### Save the model state

In [14]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

#### Prediction function

The predict function is defined to provide the flexibility of choosing the desired **top_K** character choice 
along with the canging hidden state post generation of last character

In [18]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(GPU):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(GPU):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

####  Receive the output

In [19]:
def sample(net, size, prime='The', top_k=None):
    '''
    net: The CharRNN object
    size: The number of characters required after the *prime* word
    prime: An initialising word post which the model predicts
    top_k: The degree of freedom given to choose from the top k class probabilities
    '''
        
    if(GPU):
        net.cuda()ites
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [20]:
print(sample(net, 1000, prime='Anna', top_k=4))

Anna she
would hold him to see her hands or to the country.

Telent off her hands too, he went on. She was the face of all the
complacient, waiting a commaticate this father of her families, and
showed a commans that saw the crain, the same sones, and happened on
them about him.

"You've so glad to be all this always are taken a sign of the man," he
was, and with a smile to carry him. He sat down by the creaking coansing out
his father. Her long clearly coming at the tirling, at the course was
already to blanded this he doon to the meaning of his soft one that was
not asking that the mother seemed. "I'm to say, and have you must any service
in a sort of man in the counsily interest to me on the mill of
the most matires, and he held her," said Stepan Arkadyevitch.

"Yes, I see, and say there is as that is the sight of this, and
I can't be doing before, I do not believe it in a contemptuous....
What is the chief too?" she went on. "What are you all these moment and
sometimes would never 

In [22]:
print(sample(net, 1000, prime='Anna', top_k=2))

Anna said
to him, and the same standing of her hands an instant, and the same
standing of his words, when they were something there were the
contrary of his face, and that her hands was the same side of his head
and so that he had been the simple the significant face that the
peasants, who were so such a chance of the strengs of taking out
the condition of his face, and his face was so much the same
time the conviction of her husband and her face was still, and she said
to her. His feather were so supper, and the country, with to her and the
same side, the conversation was so much that his feet and head as he
went on, and the carriage was a significant, and there, was the
same time and to this soul to the convinced of his face and her hands on
this condition of the same, and that the counters was not all abroad,
the same sign of him, as though to say that the contrary were a strange
conversation, the same simplest carriages was, and the countess and
the caresting of his wife was someth